Project setup

In [1]:
!git clone https://github.com/Mackenzie-Beck/CS3220.git

Cloning into 'CS3220'...
remote: Enumerating objects: 385, done.
remote: Counting objects: 100% (385/385), done.
remote: Compressing objects: 100% (272/272), done.
remote: Total 385 (delta 169), reused 321 (delta 105), pack-reused 0 (from 0)
Receiving objects: 100% (385/385), 1.97 MiB | 11.80 MiB/s, done.
Resolving deltas: 100% (169/169), done.


In [10]:
cd CS3220

[Errno 2] No such file or directory: 'CS3220'
/content/CS3220


In [12]:
cd lab2/

/content/CS3220/lab2


In [15]:
cd cs3220_A2/SUBMISSION/Part3

/content/CS3220/lab2/cs3220_A2/SUBMISSION/Part3


In [16]:
ls

agentClass.py     companyEnvClass.py   envProClass.py    part3.txt     test_environment.py
agentPrograms.py  directions.py        locations.py      __pycache__/  thingClass.py
agents.py         environmentClass.py  part3EnvClass.py  rules.py


In [20]:
# directions for reflex agent
# Left and right directions for reflex agent
left = "Left"
right = "Right"

In [21]:
# these are the locations for the company specified in part 3

Room1_A, Room1_B, Room1_C, Room2_A, Room2_B, Room2_C = (0,0), (0,1), (0,2), (1,0), (1,1), (1,2)

In [22]:
import random
from envProClass import environmentPro
from thingClass import Thing
from locations import *
from directions import *
from agents import Student, ITStaff,OfficeManager




class part3Env(environmentPro):
  def __init__(self):
    super().__init__()
    self.locations=[Room1_A, Room1_B, Room1_C, Room2_A, Room2_B, Room2_C]

  def default_location(self, thing):
    print("The item is starting in random location...")
    return random.choice(self.locations)




  def percept(self, agent):
    #return a list of things that are in our agent's location
    things = self.list_things_at(agent.location)
    return agent.location, agent.direction, things





  def execute_action(self, agent, action):
    #changes the state of the environment based on what the agent does.
    if self.is_agent_alive(agent):
      if action=='Go forward':
          # if right then check if agent is in rooms that require special actions
          if agent.direction==right:
            if agent.location == Room1_C:
                agent.location = Room2_C
                agent.direction = left
                agent.performance -= 1

            elif agent.location == Room2_C:
                agent.location = Room1_C
                agent.direction = left
                agent.performance -= 1
            # if not in special rooms, then move forward
            else:
                agent.location = tuple(map(sum, zip(agent.location, (0,1))))
                agent.performance -= 1
          # if left then just move forward because rules_match will take care of the special actions
          elif agent.direction==left:
             agent.location = tuple(map(sum, zip(agent.location, (0,-1))))
             agent.performance -= 1

      elif action == 'Give mail':
        items = self.list_things_at(agent.location, thingClass=OfficeManager)
        agent.performance += 3
        self.update_agent_alive(agent)
        print("The Agent decided to {} to {} at location: {}".format(action,items[0],agent.location))
        self.delete_thing(items[0])

      elif action == 'Give donuts':
        items = self.list_things_at(agent.location, thingClass=ITStaff)
        agent.performance += 3
        self.update_agent_alive(agent)
        print("The Agent decided to {} to {} at location: {}".format(action,items[0],agent.location))
        self.delete_thing(items[0])

      elif action == 'Give pizza':
        items = self.list_things_at(agent.location, thingClass=Student)
        agent.performance += 3
        self.update_agent_alive(agent)
        print("The Agent decided to {} to {} at location: {}".format(action,items[0],agent.location))
        self.delete_thing(items[0])

      elif action == 'Stop':
        agent.alive = False



      self.update_agent_alive(agent)
     # print("The Agent decided to {} at location: {}".format(action,agent.location))


In [23]:
def interpret_input_a2p3Rules(percept):
  # this is where the agent interprets its percept and updates env
  loc, dir, things = percept
  status = 'Clear'

  # if there are no things in the room, check if the agent is in the last room
  if len(things) == 0:
    if (loc in [Room1_A, Room2_A]) and dir == left:
      status = 'Last room'
  else:
    for thing in things:
      if isinstance(thing, OfficeManager):
        status = 'Office manager'
      elif isinstance(thing, ITStaff):
        status = 'IT'
      elif isinstance(thing, Student):
        status = 'Student'

  return status



def rule_match_a2p3(state, rules):
  # this is where the agent selects an action based on its state and rules
  return rules[state]

In [25]:
from part3EnvClass import part3Env
from agents import ReflexAgentP3

# This funciton runs the environment for 100 steps and prints the final performance of the agent


def test_environment():
    # Create the environment
    env = part3Env()

    # Create the agent
    agent = ReflexAgentP3()
    # Add the agent to the environment
    env.add_thing(agent)


    # Add 10 things to the environment in random locations
    from agents import Student, ITStaff, OfficeManager
    import random

    things_to_add = [
        Student(), Student(), Student(),
        ITStaff(), ITStaff(),
        OfficeManager(), OfficeManager(),
        Student(), ITStaff(), OfficeManager()
    ]

    for thing in things_to_add:
        location = env.default_location(thing)
        env.add_thing(thing, location)
        print(f"Added {type(thing).__name__} at location: {location}")










    # Run the environment for a certain number of steps
    print("Begin Simulation ----------------------------")
    for _ in range(100):  # Run for 100 steps
        env.step()
        if not agent.alive:
            break

    # Print final performance
    print(f"Agent's final performance: {agent.performance}")

if __name__ == "__main__":
    test_environment()

Welcome!
The item is starting in random location...
The item is starting in random location...
Added Student at location: (0, 0)
The item is starting in random location...
Added Student at location: (0, 0)
The item is starting in random location...
Added Student at location: (1, 2)
The item is starting in random location...
Added ITStaff at location: (0, 2)
The item is starting in random location...
Added ITStaff at location: (0, 2)
The item is starting in random location...
Added OfficeManager at location: (1, 1)
The item is starting in random location...
Added OfficeManager at location: (0, 1)
The item is starting in random location...
Added Student at location: (0, 1)
The item is starting in random location...
Added ITStaff at location: (0, 0)
The item is starting in random location...
Added OfficeManager at location: (1, 1)
Begin Simulation ----------------------------
Agent percepted ((1, 0), 'Right', []).
Agent decided to do Go forward.
Agent percepted ((1, 1), 'Right', [<OfficeM